In [25]:
import requests

r = requests.get('http://127.0.0.1:5000/v1/models')
models = [
    model['id']
    for model in r.json()['data']
]

model_name = models[0] # select first
models

['airoboros-c34b-2.1.Q5_K_S.gguf',
 'all-mpnet-base-v2',
 'gpt-3.5-turbo',
 'text-embedding-ada-002',
 'airoboros-c34b-2.1.Q5_K_S.gguf',
 'airoboros-l2-13b-2.1.Q5_K_M.gguf',
 'TheBloke_Llama-2-13B-GPTQ',
 'TheBloke_OpenAssistant-Llama2-13B-Orca-8K-3319-GPTQ',
 'TheBloke_Wizard-Vicuna-30B-Uncensored-GPTQ',
 'THUDM_chatglm2-6b',
 'THUDM_chatglm-6b',
 'wizardcoder-python-34b-v1.0.Q4_K_M.gguf']

In [27]:
import openai

openai.api_base = "http://127.0.0.1:5000/v1"

# create a chat completion
completion = openai.ChatCompletion.create(
  model=model_name,
  tempture=0.6,
  top_p=0.5,
  max_tokens=500,
  repeat_penalty=1,
  messages=[{"role": "user", "content": """As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Your response should be in JSON format.

Input: Extract 4 main points of view in the text, and Google search for each point of view in text.

Available functions:
points_analysis:
  description: his tool extract the points of view in given content.
  parameters:
    content: The phrase or document to analyze.
google_search:
  description: Help the user find information by converting the input query into a series of search terms and filters that may help pinpoint the location of the information.
  parameters:
    search_terms: List of keywords and/or phrases that are of high importance to the input.
    alternatives: List of alternative keywords and/or phrases that are of high importance to the input, which are variations of the actual input keywords/phrases. For example, acronyms, common alternate synonyms, etc.
"""}]
)
# print the completion
print(completion.choices[0].message.content)

{
  "function": "points_analysis",
  "parameters": {
    "content": "The text you want to analyze"
  }
}


In [28]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

class GoogleSearch(BaseModel):
    search_terms: List[str] = Field(description="List of keywords and/or phrases that are of high importance to the input")
    alternatives: List[str] = Field(description="List of alternative keywords and/or phrases that are of high importance to the input, " + \
                                                "which are variations of the actual input keywords/phrases. For example, acronyms, common alternate synonyms, etc")

    # You can add custom validation logic easily with Pydantic.
    @validator("search_terms")
    def search_terms_validation(cls, field):
        if len(field) > 10:
            raise ValueError("Max search terms limit reached (<=10) ")
        return field

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=GoogleSearch)

prompt = PromptTemplate(
    template="Extract the points of view in following content, into keywords and/or phrases that are of high importance to the given content.\n\nContent:\n{content}\n\n{format_instructions}\n",
    input_variables=["content"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(content="""In the world of language models, a prompt is a piece of text that instructs the model to generate a specific type of response.
A prompt template, as the name suggests, is a reproducible way to generate such prompts. It's essentially a text string that can take in a set of parameters from the end user and generate a prompt accordingly.
A prompt template can contain instructions for the language model, a set of few-shot examples to guide the model's response, and a question for the model.
In conclusion, prompt templates in LangChain are a powerful tool for generating dynamic prompts for languagemodels. They offer flexibility and control over the prompts,
allowing you to guide the model’s responses effectively. Whether you’re creating a language model for a specific task or exploring the capabilities of language models, prompt templates can be a game-changer.""")
print(_input.to_string())

Extract the points of view in following content, into keywords and/or phrases that are of high importance to the given content.

Content:
In the world of language models, a prompt is a piece of text that instructs the model to generate a specific type of response.
A prompt template, as the name suggests, is a reproducible way to generate such prompts. It's essentially a text string that can take in a set of parameters from the end user and generate a prompt accordingly.
A prompt template can contain instructions for the language model, a set of few-shot examples to guide the model's response, and a question for the model.
In conclusion, prompt templates in LangChain are a powerful tool for generating dynamic prompts for languagemodels. They offer flexibility and control over the prompts,
allowing you to guide the model’s responses effectively. Whether you’re creating a language model for a specific task or exploring the capabilities of language models, prompt templates can be a game-ch

In [35]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

openai.api_base = "http://127.0.0.1:5000/v1"
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature = 0.2)

output = model([SystemMessage(content="As an AI assistant, please select the most suitable function and parameters from the list of available functions below, based on the user's input. Provide your response in JSON format."),
                HumanMessage(content=_input.to_string())])
print(output.content)

{
  "properties": {
    "search_terms": [
      "point of view",
      "perspective",
      "opinion",
      "view"
    ],
    "alternatives": [
      "outlook",
      "standpoint",
      "stance",
      "attitude"
    ]
  }
}


In [33]:
parser.parse(output.content)

OutputParserException: Failed to parse GoogleSearch from completion {
  "properties": {
    "search_terms": [
      "points of view",
      "keywords and/or phrases that are of high importance to the given content"
    ],
    "alternatives": []
  },
  "required": ["search_terms"]
}. Got: 2 validation errors for GoogleSearch
search_terms
  field required (type=value_error.missing)
alternatives
  field required (type=value_error.missing)

In [36]:
import json

json_object = json.loads(output.content)
parser.pydantic_object.parse_obj(json_object['properties'])

GoogleSearch(search_terms=['point of view', 'perspective', 'opinion', 'view'], alternatives=['outlook', 'standpoint', 'stance', 'attitude'])